In [ ]:
# =========================
# Single-cell, optimized, compatible version
# =========================

import transformers
print("Transformers version:", transformers.__version__)

import torch
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

# -------------------------
# Device
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------
# Load dataset
# -------------------------
newsgroups = fetch_20newsgroups(
    subset="all",
    remove=("headers", "footers", "quotes")
)

texts = newsgroups.data
labels = newsgroups.target
label_names = newsgroups.target_names

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
)

# -------------------------
# Tokenizer (shorter = faster)
# -------------------------
MAX_LEN = 128

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
)

test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
)

# -------------------------
# Dataset wrapper
# -------------------------
class NewsGroupDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = NewsGroupDataset(train_encodings, train_labels)
test_dataset = NewsGroupDataset(test_encodings, test_labels)

# -------------------------
# Model
# -------------------------
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=20,
)
model.to(device)

# -------------------------
# Metrics
# -------------------------
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# -------------------------
# TrainingArguments (version-safe)
# -------------------------
import inspect

base_args = {
    "output_dir": "./results",
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 2,
    "weight_decay": 0.01,
    "logging_steps": 100,
    "report_to": "none",
}

sig = inspect.signature(TrainingArguments.__init__)
params = sig.parameters

if "evaluation_strategy" in params:
    base_args["evaluation_strategy"] = "epoch"
elif "eval_strategy" in params:
    base_args["eval_strategy"] = "epoch"

if "save_strategy" in params:
    base_args["save_strategy"] = "epoch"

training_args = TrainingArguments(**base_args)

# -------------------------
# Trainer
# -------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# -------------------------
# Train & evaluate
# -------------------------
trainer.train()
eval_results = trainer.evaluate()
print("Eval results:", eval_results)

# -------------------------
# Inference
# -------------------------
text = "The government passed a new law affecting international trade."

inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
)

inputs = {k: v.to(device) for k, v in inputs.items()}

model.eval()
with torch.no_grad():
    outputs = model(**inputs)

predicted_class = outputs.logits.argmax(dim=1).item()
print("Predicted topic:", label_names[predicted_class])


Transformers version: 4.57.3


KeyboardInterrupt: 